Import Libraries

In [1]:
import pandas as pd
import numpy as np

Extract USER-DRUG Interations

In [2]:
prec=pd.read_csv(r'...\PRESCRIPTIONS.csv.gz')
prec.head()

C:\Users\Administrator\AppData\Local\Temp\1\ipykernel_30376\558025341.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  prec=pd.read_csv(r'C:\Users\Administrator\Desktop\medllm evn\Data\mimic-iii-clinical-database-1.4\PRESCRIPTIONS.csv.gz')


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,2214776,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Tacrolimus,Tacrolimus,Tacrolimus,TACR1,021796,469061711.0,1mg Capsule,2,mg,2,CAP,PO
1,2214775,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,WARF5,006562,56017275.0,5mg Tablet,5,mg,1,TAB,PO
2,2215524,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Heparin Sodium,NaN,NaN,HEPAPREMIX,006522,338055002.0,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV
3,2216265,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,BASE,D5W,NaN,NaN,HEPBASE,NaN,0.0,HEPARIN BASE,250,ml,250,ml,IV
4,2214773,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Furosemide,Furosemide,Furosemide,FURO20,008208,54829725.0,20mg Tablet,20,mg,1,TAB,PO


In [3]:
diagnoses=pd.read_csv(r'...\DIAGNOSES_ICD.csv.gz')
diagnoses.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254


In [4]:
diagnoses_d=pd.read_csv(r'...\D_ICD_DIAGNOSES.csv.gz')
diagnoses_d.head()

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,174,01166,TB pneumonia-oth test,"Tuberculous pneumonia [any form], tubercle bac..."
1,175,01170,TB pneumothorax-unspec,"Tuberculous pneumothorax, unspecified"
2,176,01171,TB pneumothorax-no exam,"Tuberculous pneumothorax, bacteriological or h..."
3,177,01172,TB pneumothorx-exam unkn,"Tuberculous pneumothorax, bacteriological or h..."
4,178,01173,TB pneumothorax-micro dx,"Tuberculous pneumothorax, tubercle bacilli fou..."


In [11]:
# -------------------- Filter ICD-9 codes 280-285 --------------------
# Ensure ICD9_CODE is string
diagnoses['ICD9_CODE'] = diagnoses['ICD9_CODE'].astype(str)

# Keep only codes starting with '280', '281', '282', '283', '284', '285'
valid_icd9_prefixes = ['280', '281', '282', '283', '284', '285']
diagnoses_filtered = diagnoses[diagnoses['ICD9_CODE'].str[:3].isin(valid_icd9_prefixes)]

# -------------------- Get patients with these ICD-9 codes --------------------
patients_with_anemia = diagnoses_filtered['HADM_ID'].unique()

# -------------------- Filter prescriptions --------------------
prec_filtered = prec[prec['HADM_ID'].isin(patients_with_anemia)]

# -------------------- Create user-drug interaction --------------------
# Choose user = HADM_ID, item = DRUG_NAME_GENERIC
df = prec_filtered[['HADM_ID', 'DRUG_NAME_GENERIC']].dropna()

# Group by patient and drug
user_drug = df.groupby(['HADM_ID', 'DRUG_NAME_GENERIC']).size().reset_index(name='rating')

# Optional: binarize if you want implicit feedback (recommended for MF)
user_drug['rating'] = 1

print(user_drug.head())


C:\Users\Administrator\AppData\Local\Temp\1\ipykernel_30376\2836943446.py:6: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  prec = pd.read_csv(r'C:\Users\Administrator\Desktop\medllm evn\Data\mimic-iii-clinical-database-1.4\PRESCRIPTIONS.csv.gz')


   HADM_ID          DRUG_NAME_GENERIC  rating
0   100003              Acetaminophen       1
1   100003  Chloraseptic Throat Spray       1
2   100003                 FoLIC Acid       1
3   100003                 Furosemide       1
4   100003                  Lactulose       1


In [13]:
user_drug = user_drug.rename(columns={
    'HADM_ID': 'user',
    'DRUG_NAME_GENERIC': 'item'
})

print(user_drug.head())

     user                       item  rating
0  100003              Acetaminophen       1
1  100003  Chloraseptic Throat Spray       1
2  100003                 FoLIC Acid       1
3  100003                 Furosemide       1
4  100003                  Lactulose       1


In [14]:
user_drug.to_csv(r'...\user_drug_rating_visit_anemia.csv', index=False)